[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/05_Devices_and_HumanInLoop.ipynb)


In [1]:
import os
import sys
from pathlib import Path
import subprocess

def _ensure_repo():
    workspace = os.environ.get('GITHUB_WORKSPACE')
    if workspace:
        repo_root = Path(workspace).resolve()
        src_path = repo_root / 'src'
        if src_path.exists():
            sys.path.insert(0, str(src_path))
            return repo_root
    repo_root = Path().resolve()
    src_path = repo_root / 'src'
    if src_path.exists():
        sys.path.insert(0, str(src_path))
        return repo_root
    repo_dir = Path('IINTS-SDK')
    if not repo_dir.exists():
        subprocess.check_call(['git', 'clone', 'https://github.com/python35/IINTS-SDK.git'])
    repo_root = repo_dir.resolve()
    os.chdir(repo_root)
    src_path = repo_root / 'src'
    sys.path.insert(0, str(src_path))
    return repo_root

_ensure_repo()


PosixPath('/home/runner/work/IINTS-SDK/IINTS-SDK')

# Sensor/Pump Models and Human-in-the-loop

Simulate sensor noise, pump caps, and real-time interventions.


In [2]:
from pathlib import Path
import json
from iints.presets import load_presets

import iints
import yaml
from importlib import resources
from iints.core.algorithms.mock_algorithms import ConstantDoseAlgorithm
from iints.core.devices import SensorModel, PumpModel
from iints.core.simulator import Simulator
from iints.core.patient.models import PatientModel
from iints.validation import build_stress_events

presets = load_presets()
preset = next(p for p in presets if p["name"] == "baseline_t1d")

patient_config = yaml.safe_load(resources.read_text("iints.data.virtual_patients", f"{preset['patient_config']}.yaml"))
patient_model = PatientModel(**iints.validation.validate_patient_config_dict(patient_config).model_dump())

sensor = SensorModel(noise_std=1.0, bias=2.0, lag_minutes=0, dropout_prob=0.0, seed=1)
pump = PumpModel(max_units_per_step=0.5, quantization_units=0.05, dropout_prob=0.01, seed=1)

def on_step(ctx):
    # If glucose is too low, inject 12g rescue carbs
    if ctx["glucose_actual_mgdl"] < 70:
        return {"additional_carbs": 12, "note": "rescue carbs"}
    return None

simulator = Simulator(
    patient_model=patient_model,
    algorithm=ConstantDoseAlgorithm(dose=0.2),
    time_step=preset["time_step_minutes"],
    sensor_model=sensor,
    pump_model=pump,
    on_step=on_step,
    seed=42,
)

for event in build_stress_events(preset["scenario"]["stress_events"]):
    simulator.add_stress_event(event)

results, safety = simulator.run_batch(duration_minutes=240)
results[["time_minutes", "glucose_actual_mgdl", "sensor_status", "pump_status", "human_intervention"]].head()


/tmp/ipykernel_2247/2425727200.py:17: DeprecationWarning: read_text is deprecated. Use files() instead. Refer to https://importlib-resources.readthedocs.io/en/latest/using.html#migrating-from-legacy for migration advice.
  patient_config = yaml.safe_load(resources.read_text("iints.data.virtual_patients", f"{preset['patient_config']}.yaml"))
Simulation terminated early: Critical failure: glucose < 40.0 mg/dL for 30 minutes.


,time_minutes,glucose_actual_mgdl,sensor_status,pump_status,human_intervention
0,0,140.000000,ok,ok,False
1,5,118.977778,ok,ok,False
2,10,101.086667,ok,ok,False
3,15,85.857000,ok,ok,False
4,20,72.889561,ok,ok,False
